In [624]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize

# Compile all our data into one dataframe

In [625]:
#Import our 3 csvs containing the data from SQL queries and Census
df_hpd = pd.read_csv('data/hpd_complaintsbyzip.csv')
df_dob = pd.read_csv('data/dobjob_byzip.csv')
df_income = pd.read_csv('data/census_income_12_16.csv')

In [626]:
#Merge income and dob dataframes
df_inc_dob = pd.merge(df_income, df_dob, left_on=['zip', 'year'], right_on=['zip', 'year'])

In [627]:
#Merge the above DF with the HPD complaints
df_final = pd.merge(df_inc_dob, df_hpd, left_on=['zip', 'year'], right_on=['zip', 'year'])
df_final.sort_values(['zip', 'year'])

,zip,year,income,dobjob_count,hpd_complaints_count
5,10026,2013,43107,277,1
14,10026,2014,47318,372,1134
24,10026,2015,47264,415,2574
0,10027,2012,35694,766,1
6,10027,2013,37872,864,4
15,10027,2014,40013,945,2113
25,10027,2015,40782,1022,4266
34,10027,2016,42754,1004,3624
1,10030,2012,30674,181,1
7,10030,2013,31925,199,1


In [628]:
#Make a list of each zipcode in the DF for our for loop
zip_list = df_final['zip'].unique().tolist()
zip_list

[10027, 10030, 10031, 10032, 10033, 10026, 10037, 10039, 10040, 10034]

In [629]:
#Create our dataframe with all info, plus the percent change between each
#make a blank DF
df_out = pd.DataFrame()

for zip_code in zip_list:
    #Make a temporary df that contains only the info from one zip code
    df_zip = df_final[df_final['zip'] == zip_code]
    #calculate the percent change in the three values we're interested in: income, DOB Job filings and HPD complaints
    df_zip['income_change'] = df_zip['income'].pct_change().values*100
    df_zip['dobjob_change'] = df_zip['dobjob_count'].pct_change().values*100
    df_zip['hpd_complaints_change'] = df_zip['hpd_complaints_count'].pct_change().values*100
    #concat this new information together into one df
    frames = [df_out, df_zip]
    df_out = pd.concat(frames)

/Users/sabi/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/sabi/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/sabi/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

# Take our completed data set and prepare it for leaflet

In [630]:
def group_year(year):
    if year == 2012:
        group_id = 0
    elif year == 2013:
        group_id = 1
    elif year == 2014:
        group_id = 2
    elif year == 2015:
        group_id = 3
    elif year == 2016:
        group_id = 4
    return group_id

In [710]:
#A function that compares the percent change in income of a neighborhood and assigns it a value from dark orange,
# for loss in income, to yellow for the largest raises in income. 

def color_picker(income_change): 
    #Biggest decrease
    color = ''
    if income_change < 0:
        color = '#FF2D1D'
    #Lower increase
    elif 0 < income_change < 1.8:
        color = '#ff9810'
    #Middle increase
    elif 1.8 < income_change < 3:
        color = '#ffb50b'
    #Higher increase
    elif 3 < income_change < 5:
        color = '#ffc808'
    elif 5 < income_change < 9:
        color = '#ffdb05'        
    #Biggest increase
    elif income_change > 8:
        color = '#FFF800'
    #everthing else is gray
    else: 
        color = '#d3d3d3'
    return color

In [711]:
#Set group name to year
df_out['properties.group_name'] = df_out['year']
#set article to a sencene
df_out['properties.article'] = "<p>Income in " + df_out['zip'].map(str) + " was $" + df_out['income'].map(str) + " and there were " + df_out['dobjob_count'].map(str) + " DOB Job filings and " + df_out['hpd_complaints_count'].map(str) + " 311 complaints</p>"
#set headline to a sentence
df_out['properties.headline'] = "<p>" + df_out['zip'].map(str) + " saw an increase in income of " + df_out['income_change'].map(str) + "%. The number of 311 complaints filed increased by " + df_out['hpd_complaints_change'].map(str) + "% and the number of DOB Job filings increased by " + df_out['dobjob_change'].map(str) +"%</p>"
#set name to zipcode
df_out['properties.name'] = df_out['zip'].map(str)
#set group id to a value between 1 and 5
df_out['properties.group_id'] = df_out.year.apply(lambda x: group_year(x))
#picks a color based in change in income
df_out['properties.color'] = df_out.income_change.apply(lambda x: color_picker(x))

In [712]:
#remove the old columns and clean the dataframe
output = df_out.drop(df_out.columns[[0,1,2,3,4,5,6,7]], axis=1)
output

,properties.group_name,properties.article,properties.headline,properties.name,properties.group_id,properties.color
0,2012,<p>Income in 10027 was $35694 and there were 7...,<p>10027 saw an increase in income of nan%. Th...,10027,0,#d3d3d3
6,2013,<p>Income in 10027 was $37872 and there were 8...,<p>10027 saw an increase in income of 6.101865...,10027,1,#ffdb05
15,2014,<p>Income in 10027 was $40013 and there were 9...,<p>10027 saw an increase in income of 5.653253...,10027,2,#ffdb05
25,2015,<p>Income in 10027 was $40782 and there were 1...,<p>10027 saw an increase in income of 1.921875...,10027,3,#ffb50b
34,2016,<p>Income in 10027 was $42754 and there were 1...,<p>10027 saw an increase in income of 4.835466...,10027,4,#ffc808
1,2012,<p>Income in 10030 was $30674 and there were 1...,<p>10030 saw an increase in income of nan%. Th...,10030,0,#d3d3d3
7,2013,<p>Income in 10030 was $31925 and there were 1...,<p>10030 saw an increase in income of 4.078372...,10030,1,#ffc808
16,2014,<p>Income in 10030 was $32561 and there were 2...,<p>10030 saw an increase in income of 1.992169...,10030,2,#ffb50b
26,2015,<p>Income in 10030 was $33196 and there were 3...,<p>10030 saw an increase in income of 1.950185...,10030,3,#ffb50b
35,2016,<p>Income in 10030 was $33720 and there were 2...,<p>10030 saw an increase in income of 1.578503...,10030,4,#ff9810


In [713]:
#import geometry from NYC opendata
with open('zipsgeo.json') as json_data:
    geometry_data = json.load(json_data)

In [714]:
#Put the json in a DF
df_geo = pd.DataFrame.from_dict(json_normalize(geometry_data['features']), orient='columns')

In [715]:
#Drop useless rows
out_geometry = df_geo.drop(df_geo.columns[[2,3,4,5,6,7,8,9,10,11]], axis=1)


In [716]:
#merge with output on zipcode
output = output.merge(out_geometry, left_on='properties.name', right_on='properties.postalCode')
output.rename(columns={'properties.ZIPCODE': 'properties.name', 'properties.POPULATION': 'properties.population'}, inplace=True)

# Create GEOJson

In [717]:
output = output.drop(output.columns[[3]], axis=1)
output.reset_index()
ok_json = json.loads(output.to_json(orient='records'))

In [718]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data

In [719]:
geo_format = process_to_geojson(ok_json)

In [720]:
with open('geo-data.js', 'w') as outfile:
    outfile.write("var infoData = ")
with open('geo-data.js', 'a') as outfile:
    json.dump(geo_format, outfile)